In [1]:
!cp /content/drive/MyDrive/datasets/kern_scores/schubert.zip /content
!cp /content/drive/MyDrive/datasets/kern_scores/encoded_foster.zip /content
!cp /content/drive/MyDrive/py_modules/lstm/train.py /content
!cp /content/drive/MyDrive/py_modules/lstm/song_encoder.py /content
!cp /content/drive/MyDrive/py_modules/lstm/song_loader.py /content
!cp /content/drive/MyDrive/py_modules/lstm/song_preprocess.py /content
!cp /content/drive/MyDrive/py_modules/lstm/transposer.py /content
!cp /content/drive/MyDrive/py_modules/lstm/generation.py /content

In [2]:
import os
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
import music21 as m21

from song_loader import SongLoader 
from song_encoder import SongEncoder 
from transposer import Transposer
from song_preprocess import create_mapping, cast_to_int, generate_train_sequences
from train import build_lstm
from generation import MelodyGenerator

In [3]:
with ZipFile('schubert.zip') as z:
    z.extractall()

with ZipFile('encoded_foster.zip') as z:
    z.extractall()

In [4]:
loader1 = SongLoader(input_format='krn')
transposer = Transposer()
encoder = SongEncoder(time_step=0.25)

schubert, _ = loader1.load('schubert')

In [5]:
foster = []
for root, _, files in os.walk('/content/encoded_foster'):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r') as op:
            song = op.read()
            foster.append(song)

In [6]:
songs = []
for schu in schubert:
    transposed = transposer.transpose(schu)
    encoded = encoder.encode(transposed)
    songs.append(encoded)

for fos in foster:
    songs.append(fos)

In [7]:
songs = encoder.create_string(songs, sequence_length=64)
int_map = create_mapping(songs)
songs = cast_to_int(songs, int_map)
inputs, targets = generate_train_sequences(songs, sequence_length=64)

In [23]:
def train_test_split(inputs, targets, train_size=0.8):
    n = len(inputs)
    indxs = np.arange(n)
    np.random.shuffle(indxs)
    split_indx = int(train_size * n)
    inputs = inputs[indxs]
    targets = targets[indxs]

    x_train, x_test = inputs[:split_indx], inputs[split_indx:]
    y_train, y_test = targets[:split_indx], targets[split_indx:]

    return x_train, y_train, x_test, y_test

In [24]:
x_train, y_train, x_test, y_test = train_test_split(inputs, targets)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(25508, 64, 40) (6378, 64, 40) (25508,) (6378,)


In [ ]:
nodes = np.random.randint(low=100, high=300)
lr = np.random.uniform(low=0.0001, high=0.01)
b = np.random.randint(low=16, high=64)
e = np.random.randint(low=50, high=100)

print(f'{nodes} nodes')
print(f'learning rate = {lr}')
print(f'batch size: {b}')
print(f'Epochs: {e}')

179 nodes
learning rate = 0.0024128836230915985
batch size: 25
Epochs: 63


In [8]:
out_units = max(int_map.values())+1
print(out_units)

40


In [25]:
model = build_lstm(nodes=[184], output_units=out_units, loss='sparse_categorical_crossentropy', learning_rate=0.0028)

In [ ]:
model.fit(x_train, y_train, epochs=52, batch_size=62)

In [27]:
prediction = model.predict(x_test)
print(accuracy_score(np.argmax(prediction, axis=1), y_test))

0.8386641580432738


In [ ]:
lstm_dir = '/content/drive/MyDrive/models/lstm/generative.h5'
model.save(lstm_dir)

In [9]:
model = build_lstm(nodes=[184], output_units=out_units, loss='sparse_categorical_crossentropy', learning_rate=0.0028)
model.load_weights('/content/drive/MyDrive/models/lstm/generative.h5')

In [28]:
seed = '60 _ _ _ 62 _ 64 _ 64 _ _ _ 62 _ _ _'
generator = MelodyGenerator(model, int_map, seq_length=64)
melody = generator.generate(seed, steps=500, max_seq_length=64, temperature=0.5)

In [30]:
generator.save_melody(melody)